In [1]:
import glob
import numpy as np

data_dir = "../data/asl/images/"


In [4]:
import tensorflow as tf

img_xy = glob.glob(data_dir + "*xy.jpg")
img_yz = glob.glob(data_dir + "*yz.jpg")

def load(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(32, 32))
    return img

In [14]:


a = np.random.random((4000, 150, 5))
b = np.split(a, 5, axis=2)

# i2 = tf.data.Dataset.from_tensor_slices(a)

inputs = tf.data.Dataset.from_tensor_slices((img_xy, img_xy, *b)) \
    .map(lambda x, y, *z: (load(x), load(y), z))

labels = tf.data.Dataset.from_tensor_slices(img_xy)
ds = tf.data.Dataset.zip((inputs, labels)).batch(4)

ds.element_spec

((TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None),
  (TensorSpec(shape=(None, 150, 1), dtype=tf.float64, name=None),
   TensorSpec(shape=(None, 150, 1), dtype=tf.float64, name=None),
   TensorSpec(shape=(None, 150, 1), dtype=tf.float64, name=None),
   TensorSpec(shape=(None, 150, 1), dtype=tf.float64, name=None),
   TensorSpec(shape=(None, 150, 1), dtype=tf.float64, name=None))),
 TensorSpec(shape=(None,), dtype=tf.string, name=None))

In [15]:
import tensorflow as tf

inputs = []

for i in range(2):
    inputs.append(tf.keras.layers.Input(shape=(32, 32, 3)))

for i in range(5):
    inputs.append(tf.keras.layers.Input(shape=(150, 1)))

x = tf.keras.layers.concatenate(inputs[:2])
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(3)(x)

model = tf.keras.models.Model(inputs, x)

model.compile()

model.predict(ds)

array([[-0.20691922,  1.2134912 , -0.02880372],
       [ 0.12261689,  1.9814444 , -0.7860656 ],
       [-0.0039759 ,  2.1413326 , -0.507591  ],
       ...,
       [-0.03184342,  1.2674892 ,  0.65227956],
       [-0.08735511,  1.1800733 , -0.27915913],
       [-0.24329877,  1.3660676 ,  0.14775828]], dtype=float32)